20240628 뱀 로봇 자유 공간에서 동역학 분석을 위한 코드

* 필요 패키지 사용

In [2]:
import numpy as np
import mujoco
import mujoco.viewer
import mediapy as media

import time
import serpenoid
import serpenoid_gamma

* 시뮬레이션 정의 파일 불러오기

In [3]:
snake = mujoco.MjModel.from_xml_path("./horcrux_sand.xml")
data = mujoco.MjData(snake)
renderer = mujoco.Renderer(snake, 720, 1280)

* Gait 궤적 사전 생성하기

In [4]:
p_c = (45, 45, 154, 154, 55, 55, 0, 0.05) 
p_m = (45, 45, 154, 154, 55, 55, 0, 0.05) 
gamma = 0

gait = serpenoid_gamma.Gait(tuple(p_c), tuple(p_m), gamma)
q = gait.CurveFunction
expand_q = np.repeat(q, 10, axis=1)

* 궤적 확인

In [5]:
expand_q.shape

(14, 2000)

* 시뮬레이션 진행 (영상 렌더링)

In [6]:
frames = []
mujoco.mj_forward(snake, data)

for i in range(expand_q.shape[1]):
    index = np.nonzero(expand_q[:, i])
    for idx in index:
        data.ctrl[idx] = expand_q[idx, i]

    mujoco.mj_step(snake, data)
    renderer.update_scene(data)
    pixel = renderer.render()

    frames.append(pixel)

media.show_video(frames, fps=200)

This browser does not support the video tag.


* 시뮬레이션 진행 (Passive viewer)

In [11]:
mujoco.mj_resetData(snake, data)
mujoco.mj_forward(snake, data)

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_COM] = True

with mujoco.viewer.launch_passive(snake, data) as viewer:
    # Viewer Options
    viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_COM] = True
    for i in range(expand_q.shape[1]):
        time_step = time.time()
        index = np.nonzero(expand_q[:, i])
        for idx in index:
            data.ctrl[idx] = expand_q[idx, i]

        mujoco.mj_step(snake, data)

        viewer.sync()


        while snake.opt.timestep - (time.time() - time_step) > 0:
            time.sleep(0)